In [6]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import silhouette_samples

def compare_clusters_using_means(
    dfs,
    cluster_labels=None,
    alpha=0.05,
    drop_cols=("file_name", "cluster"),
    include_std_n=True,
):
    if cluster_labels is None:
        cluster_labels = [f"c{i}" for i in range(len(dfs))]
    if len(cluster_labels) != len(dfs):
        raise ValueError("cluster_labels must be same length as dfs")

    numeric_features = dfs[0].select_dtypes(include=[np.number]).columns.tolist()
    numeric_features = [f for f in numeric_features if f not in drop_cols]

    rows = []
    for feature in numeric_features:
        feature_means = []
        feature_stds = []
        for df in dfs:
            mean_val = df[feature].dropna().mean()
            std_val = df[feature].dropna().std()
            feature_means.append(mean_val)
            feature_stds.append(std_val)

        arrays = [df[feature].dropna().to_numpy() for df in dfs]

        valid = [a for a in arrays if len(a) > 0]
        if len(valid) < 2:
            continue

        if all(np.nanstd(a) == 0 for a in valid):
            continue

        f_stat, p_value = stats.f_oneway(*valid)

        ns = [int(len(a)) for a in arrays]

        means_arr = np.array(feature_means, dtype=float)
        max_idx = int(np.nanargmax(means_arr))
        min_idx = int(np.nanargmin(means_arr))
        mean_difference = float(means_arr[max_idx] - means_arr[min_idx])

        row = {
            "feature": feature,
            "difference": mean_difference,
            "p_value": float(p_value),
            "significant": bool(p_value < alpha),
            "max_cluster": cluster_labels[max_idx],
            "min_cluster": cluster_labels[min_idx],
        }

        rows.append(row)

    out = pd.DataFrame(rows).sort_values(by="difference", ascending=False)
    return out


def get_silhouette_scores(dfs, cluster_labels=None, drop_cols=("file_name", "cluster")):
    if cluster_labels is None:
        cluster_labels = [f"c{i}" for i in range(len(dfs))]
    if len(cluster_labels) != len(dfs):
        raise ValueError("cluster_labels must be same length as dfs")

    numeric_features = dfs[0].select_dtypes(include=[np.number]).columns.tolist()
    numeric_features = [f for f in numeric_features if f not in drop_cols]

    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df_clean = combined_df.dropna(subset=numeric_features)

    X = combined_df_clean[numeric_features].to_numpy()
    labels = combined_df_clean["cluster"].to_numpy()

    silhouette_vals = silhouette_samples(X, labels)

    combined_df_clean = combined_df_clean.copy()
    combined_df_clean["silhouette_score"] = silhouette_vals
    
    cluster_mean_silhouette = combined_df_clean.groupby("cluster")["silhouette_score"].mean().to_dict()
    combined_df_clean["cluster_mean_silhouette"] = combined_df_clean["cluster"].map(cluster_mean_silhouette)

    return combined_df_clean[["file_name", "cluster", "silhouette_score", "cluster_mean_silhouette"]]


if __name__ == "__main__":
    # Load the dataframes
    df0 = pd.read_csv("../visuals/rep_samples/UMAP-HDBSCAN/-1/cluster_-1.csv")
    df1 = pd.read_csv("../visuals/rep_samples/UMAP-HDBSCAN/0/cluster_0.csv")
    df2 = pd.read_csv("../visuals/rep_samples/UMAP-HDBSCAN/1/cluster_1.csv")
    df3 = pd.read_csv("../visuals/rep_samples/UMAP-HDBSCAN/2/cluster_2.csv")
    df4 = pd.read_csv("../visuals/rep_samples/UMAP-HDBSCAN/3/cluster_3.csv")
    df5 = pd.read_csv("../visuals/rep_samples/UMAP-HDBSCAN/4/cluster_4.csv")
    df6 = pd.read_csv("../visuals/rep_samples/UMAP-HDBSCAN/5/cluster_5.csv")
    df7 = pd.read_csv("../visuals/rep_samples/UMAP-HDBSCAN/6/cluster_6.csv")

    dfs = [df0, df1, df2, df3, df4, df5, df6, df7]
    labels = ["-1", "0", "1", "2", "3", "4", "5", "6"]

    differences = compare_clusters_using_means(
        dfs,
        cluster_labels=labels,
        alpha=0.05,
        drop_cols=("file_name", "cluster"),
        include_std_n=True,
    )

    out_path = "../visuals/rep_samples/UMAP-HDBSCAN/cluster_differences_with_means.csv"
    differences.to_csv(out_path, index=False)

    out_path_silhouette = "../visuals/rep_samples/UMAP-HDBSCAN/silhouette_scores.csv"
    silhouette_df = get_silhouette_scores(dfs, cluster_labels=labels, drop_cols=("file_name", "cluster"))
    silhouette_df.to_csv(out_path_silhouette, index=False)

    # Show the top 15 most separating features
    print(differences.head(15)[["feature", "difference", "p_value", "max_cluster", "min_cluster"]])
    print(f"\nSaved: {out_path}")

                    feature   difference        p_value max_cluster  \
16   a1_spectral_centroid_x  1386.847228   6.198646e-65           0   
49   a1_spectral_centroid_y  1386.847228   6.198646e-65           0   
1    a0_spectral_centroid_x  1221.656083   2.137690e-56           5   
34   a0_spectral_centroid_y  1221.656083   2.137690e-56           5   
36  a0_spectral_bandwidth_y   699.735351   3.119813e-29           6   
3   a0_spectral_bandwidth_x   699.735351   3.119813e-29           6   
18  a1_spectral_bandwidth_x   609.142727   5.602666e-18           6   
51  a1_spectral_bandwidth_y   609.142727   5.602666e-18           6   
59  a1_sum_peak_magnitude_y    17.200910   8.628715e-31           2   
26  a1_sum_peak_magnitude_x    17.200910   8.628715e-31           2   
11  a0_sum_peak_magnitude_x    16.309243   2.471746e-37           2   
44  a0_sum_peak_magnitude_y    16.309243   2.471746e-37           2   
10    a0_peaks_per_second_x    12.752774  3.023947e-124           2   
43    